In [1]:
import flow
import hoomd
import numpy as np
import signac

cpu = hoomd.device.CPU()
# Initialize the HOOMD-blue execution context
#hoomd.context.initialize("")

project_root = "projects/tutorial-signac-flow-hoomd"


class MyProject(flow.FlowProject):
    pass

/Users/ychihmei/opt/anaconda3/envs/RS/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from math import ceil


def init(N):
    with hoomd.Simulation(device=cpu, seed=job.statepoint.seed):
        n = ceil(pow(N, 1 / 3))
        assert n ** 3 == N
        hoomd.init.create_lattice(unitcell=hoomd.lattice.sc(a=1.0), n=n)
        hoomd.dump.gsd("init.gsd", period=None, group=hoomd.group.all())


def sample_lj(N, sigma, seed, kT, tau, p, tauP, steps, r_cut):
    from hoomd import md

    hoomd.init.read_gsd("init.gsd", restart="restart.gsd")
    group = hoomd.group.all()
    hoomd.dump.gsd("restart.gsd", truncate=True, period=100, phase=0, group=group)
    lj = md.pair.lj(r_cut=r_cut, nlist=md.nlist.cell())
    lj.pair_coeff.set("A", "A", epsilon=1.0, sigma=sigma)
    md.integrate.mode_standard(dt=0.005)
    md.integrate.npt(group=group, kT=kT, tau=tau, P=p, tauP=tauP)
    hoomd.analyze.log("dump.log", ["V"], 100, phase=0)
    hoomd.run_upto(steps)


In [3]:
@MyProject.label
def estimated(job):
    return "V" in job.document


@MyProject.label
def sampled(job):
    return job.document.get("sample_step", 0) >= 5000


@MyProject.operation
@MyProject.post(estimated)

def estimate(job):
    job.document["V"] = job.sp.N * job.sp.kT / job.sp.p
    #sp = job.statepoint()
    #job.document["V"] = sp["N"] * sp["kT"] / sp["p"]


@MyProject.operation
@MyProject.post(sampled)
def sample(job):
    import hoomd

    with job:
        with hoomd.Simulation(device=cpu, seed=job.statepoint.seed):
            try:
                sample_lj(steps=5000, **job.statepoint())
            finally:
                job.document["sample_step"] = hoomd.get_step()

In [4]:
project = MyProject.init_project(name="FlowTutorialHOOMDProject", root=project_root)

# Uncomment the following two lines if you want to start over!
# for job in project:
#     job.remove()

for p in np.linspace(0.5, 5.0, 10):
    sp = dict(N=512, sigma=1.0, seed=42, kT=1.0, p=p, tau=1.0, tauP=1.0, r_cut=2.5)
    job = project.open_job(sp).init()
    with job:
        hoomd.Simulation(device=sp["N"],seed=None)

/Users/ychihmei/opt/anaconda3/envs/RS/lib/python3.10/site-packages/signac/synced_collections/numpy_utils.py:46: NumpyConversionWarning: Any numpy types provided will be transparently converted to the closest base Python equivalents.
  warnings.warn(NUMPY_CONVERSION_WARNING, NumpyConversionWarning)


In [7]:
project.print_status(detailed=True, parameters=["p","V"])

Fetching labels: 100%|██████████| 10/10 [00:00<00:00, 39568.91it/s]


Overview: 10 jobs/aggregates, 10 jobs/aggregates with eligible operations.

label      ratio
---------  --------------------------------------------------------
estimated  |████                                    | 1/10 (10.00%)

operation      number of eligible jobs  submission status
-----------  -------------------------  -------------------
sample                              10  [U]: 10
estimate                             9  [U]: 10

Detailed View:

job id                            operation       p  V     labels
--------------------------------  ------------  ---  ----  ---------
18c063206c37ab03d06de130c6ae2b70  estimate [U]  1    None
                                  sample [U]    1    None
2a05a487f254db90b08f576d199a83b7  sample [U]    5    None  estimated
4ef567c185064a4b27b73f62124991c0  estimate [U]  3    None
                                  sample [U]    3    None
5b3c8f0fbd9458a304222d83bb82bc30  estimate [U]  3.5  None
                                  sample [U]

In [7]:
print(V)

NameError: name 'V' is not defined

In [6]:
project.run()

UserOperationError: An exception was raised during operation sample for job or aggregate with id 2a05a487f254db90b08f576d199a83b7.

NameError: name 'V' is not defined